In [19]:
import pandas as pd

df = pd.read_csv("../../data/cf_interactions.csv")

df.head()

,user_id,product_id,rating
0,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...",B07JW9H4J1,4.2
1,"AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...",B098NS6PVG,4.0
2,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...",B096MSW6CT,3.9
3,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...",B08HDJ86NZ,4.2
4,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...",B08CF3B7N1,4.2


In [20]:
from collections import defaultdict

def build_user_items(df):
    user_items = defaultdict(set)
    for u, i in zip(df["user_id"], df["product_id"]):
        user_items[u].add(i)
    return user_items


In [21]:
def recall_at_k(model, train_df, test_df, all_items, k=10):
    train_user_items = build_user_items(train_df)
    test_user_items  = build_user_items(test_df)

    recalls = []

    for user in test_user_items:
        if user not in train_user_items:
            continue

        seen = train_user_items[user]
        true = test_user_items[user]

        if len(true) == 0:
            continue

        candidates = list(all_items - seen)
        scores = [(i, model.predict(user, i)) for i in candidates]

        scores.sort(key=lambda x: x[1], reverse=True)
        top_k = {i for i, _ in scores[:k]}

        recalls.append(len(top_k & true) / len(true))

    return sum(recalls) / len(recalls)


In [22]:
import joblib

model = joblib.load("../../ml_service/app/models/collaborative.joblib")


In [23]:
train_df = df[df["split"] == "train"]
test_df  = df[df["split"] == "test"]


KeyError: 'split'

In [ ]:
all_items = set(df["product_id"].unique())

recall_10 = recall_at_k(
    model,
    train_df,
    test_df,
    all_items,
    k=10
)

print("Recall@10:", recall_10)


NameError: name 'train_df' is not defined